In [5]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file in the current directory
load_dotenv()

# Example: Access an environment variable
# my_var = os.getenv('MY_VARIABLE')
# print(my_var)
import langgraph
print(dir(langgraph))

['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_internal', 'cache', 'channels', 'checkpoint', 'config', 'constants', 'errors', 'graph', 'managed', 'pregel', 'runtime', 'store', 'types', 'typing', 'warnings']


In [ ]:
from langgraph_app import app_graph

# Example: run the LangGraph workflow
images = ["images/page1.jpg", "images/page2.jpg", "images/page3.jpg", "images/page4.jpg"]  # Replace with your actual image file paths
markdown = app_graph.run_node("OCR", images)
print(markdown)
diagnoses = app_graph.run_node("ExtractDiagnoses", markdown)
medications = app_graph.run_node("ExtractMedications", markdown, diagnoses)
fixed_medications = app_graph.run_node("FixMedications", markdown, diagnoses, medications)
html_summary = app_graph.run_node("AddSummaryPills", markdown, diagnoses, fixed_medications)
print(html_summary)


KeyboardInterrupt: 